In [1]:
!pip install -q monai

In [232]:
!git clone https://github.com/facebookresearch/segment-anything.git

Cloning into 'segment-anything'...
remote: Enumerating objects: 295, done.
remote: Total 295 (delta 0), reused 0 (delta 0), pack-reused 295
Receiving objects: 100% (295/295), 18.30 MiB | 38.09 MiB/s, done.
Resolving deltas: 100% (155/155), done.


In [31]:
%cd /scratch/jgr7704/neuro/segment-anything/
# 

/scratch/jgr7704/neuro/segment-anything


In [32]:
ls -a


./          .ipynb_checkpoints/  assets/     segment_anything/
../         CODE_OF_CONDUCT.md   demo/       segment_anything.egg-info/
.flake8     CONTRIBUTING.md      linter.sh*  setup.cfg
.git/       LICENSE              notebooks/  setup.py
.gitignore  README.md            scripts/


In [33]:
!pip install -e .

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///scratch/jgr7704/neuro/segment-anything
  Preparing metadata (setup.py) ... done
  Attempting uninstall: segment-anything
    Found existing installation: segment-anything 1.0
    Uninstalling segment-anything-1.0:
      Successfully uninstalled segment-anything-1.0
  Running setup.py develop for segment-anything


In [35]:
import numpy as np
import matplotlib.pyplot as plt
import os

join = os.path.join
from tqdm import tqdm
from skimage import transform
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import monai
from segment_anything import sam_model_registry
import torch.nn.functional as F
import argparse
import random
from datetime import datetime
import shutil
import glob
from os import listdir
from os.path import isfile, join
import pandas as pd
from PIL import Image

# set seeds

torch.cuda.empty_cache()

In [6]:
%cd ..


/scratch/jgr7704/neuro


In [7]:
# %ls -a

In [2]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [3]:
import monai

In [10]:
def resize(path):
  dirs = os.listdir( path )
  for item in tqdm(dirs):
    if os.path.isfile(path+item):
      im = Image.open(path+item)
      f, e = os.path.splitext(path+item)
      imResize = im.resize((1024,1024), Image.NEAREST)
      imResize.save(f+e, 'PNG', quality=100)

label_path =  "/scratch/jgr7704/neuro/dataset/axon/"
output_features_path = "/scratch/jgr7704/neuro/dataset/image/"
resize(label_path)

100%|██████████| 242/242 [00:04<00:00, 57.98it/s]


In [11]:
ids=[]
label_filenames = [f for f in listdir(label_path) if isfile(join(label_path, f))]
feature_filenames = [f for f in listdir(output_features_path) if isfile(join(output_features_path, f))]
for i in range(len(feature_filenames)):
    
    # print(feature_filenames[i])
    ids.append(feature_filenames[i])
# print(ids)

df = pd.DataFrame(ids ,columns=["file_ids"])
df.to_csv('/scratch/jgr7704/neuro/full_file_ids.csv', index=False)

#sanity check
df = pd.read_csv('/scratch/jgr7704/neuro/full_file_ids.csv')
df.head()

,file_ids
0,228.png
1,27.png
2,175.png
3,225.png
4,172.png


In [12]:
import pandas as pd
import os
import cv2

df = pd.read_csv('/scratch/jgr7704/neuro/full_file_ids.csv')
ids = df['file_ids'].tolist()
non_empty_ids = []
# print(ids)
for file_id in ids:
    # print(f"--------------{label_path}----------",)
    mask_path = os.path.join(label_path, file_id)
    mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
    if cv2.countNonZero(mask) > 0:
        non_empty_ids.append(file_id)

df_non_empty = pd.DataFrame(non_empty_ids, columns=["file_ids"])
df_non_empty.sort_values(by='file_ids', inplace=True)  # Sort the DataFrame by 'file_ids'
df_non_empty.to_csv('/scratch/jgr7704/neuro/file_ids.csv', index=False)


dif = pd.read_csv('/scratch/jgr7704/neuro/file_ids.csv')
dif.head(15)

,file_ids
0,1.png
1,10.png
2,100.png
3,101.png
4,102.png
5,103.png
6,104.png
7,105.png
8,106.png
9,107.png


In [13]:
        # self.img_path = "/scratch/jgr7704/neuro/images"
        # self.mask_path = "/scratch/jgr7704/neuro/axon"

In [4]:
class SegmentationDataset(Dataset):
    def __init__(self, csv_file, bbox_shift=20):
        self.df = pd.read_csv(csv_file)
        self.ids = self.df["file_ids"]
        self.img_path = "/scratch/jgr7704/neuro/dataset/image"
        self.mask_path = "/scratch/jgr7704/neuro/dataset/axon"
        self.bbox_shift = bbox_shift
        print(f"number of images: {len(self.ids)}")

    def __len__(self):
        return len(self.ids)

    def __getitem__(self, index):
        # Load image and mask using the ID from the CSV
        img_name = f"{self.ids[index]}"
        mask_name = f"{self.ids[index]}"
     
        # Load and resize image to 1024x1024, then convert to RGB
        img = Image.open(join(self.img_path, img_name)).resize((1024, 1024)).convert("RGB")
        img = np.array(img)  # Convert image to numpy array

        img = img / 255.0

        # Load and resize mask to 1024x1024
        mask = Image.open(join(self.mask_path, mask_name)).resize((1024, 1024))
        mask = np.array(mask)  # Convert mask to numpy array

        # Convert the shape to (3, H, W) for image and (1, H, W) for mask
        img = np.transpose(img, (2, 0, 1))
        mask = np.expand_dims(mask, axis=0)  # Add an extra dimension for the channel

        label_ids = np.unique(mask)[1:]
        mask_binary = np.uint8(mask == random.choice(label_ids.tolist()))#.reshape((1024, 1024))
        mask_binary = mask_binary[0]
        # mask_binary = np.uint8(mask == random.choice(label_ids.tolist()))[1]  # only one label, (1024, 1024)

        # print(mask_binary.shape)
        y_indices, x_indices = np.where(mask_binary > 0)
        x_min, x_max = np.min(x_indices), np.max(x_indices)
        y_min, y_max = np.min(y_indices), np.max(y_indices)
        # add perturbation to bounding box coordinates
        H, W = mask_binary.shape
        x_min = max(0, x_min - random.randint(0, self.bbox_shift))
        x_max = min(W, x_max + random.randint(0, self.bbox_shift))
        y_min = max(0, y_min - random.randint(0, self.bbox_shift))
        y_max = min(H, y_max + random.randint(0, self.bbox_shift))
        bboxes = np.array([x_min, y_min, x_max, y_max])

        return (
            torch.tensor(img).float(),
            torch.tensor(mask_binary[None, :, :]).long(),
            torch.tensor(bboxes).float(),
            img_name,
        )

In [15]:
# !wget https://drive.google.com/drive/folders/1ETWmi4AiniJeWOt6HAsYgTjYv_fkgzoN?usp=drive_link

In [65]:
%mkdir /scratch/jgr7704/neuro/checkpoint_save
%cd ..

mkdir: cannot create directory ‘/scratch/jgr7704/neuro/checkpoint_save’: File exists
/scratch/jgr7704/neuro


In [64]:
pwd

'/scratch/jgr7704/neuro/segment-anything'

In [66]:
lr=0.0005
batch_size = 1
data_path = "/scratch/jgr7704/neuro/dataset"
checkpoint = "/scratch/jgr7704/neuro/medsam_vit_b.pth"
model_type = "vit_b"
work_dir = "/scratch/jgr7704/neuro/checkpoint_save"
num_epochs = 2
num_workers=6
use_wandb = 1
use_amp = 0
resume = ""
task_name = "euroSAM-ViT-"
num_epochs = num_epochs
iter_num = 0
start_epoch = 0
losses = []
best_loss = 1e10

In [67]:
os.makedirs(work_dir, exist_ok=True)

In [68]:
# Instantiate your dataset
tr_dataset = SegmentationDataset(csv_file='file_ids.csv',)
tr_dataloader = DataLoader(tr_dataset, batch_size=batch_size, shuffle=True)

def show_mask(mask, ax, random_color=False):
    if random_color:
        color = np.concatenate([np.random.random(3), np.array([0.6])], axis=0)
    else:
        color = np.array([251 / 255, 252 / 255, 30 / 255, 0.6])
    h, w = mask.shape[-2:]
    mask_image = mask.reshape(h, w, 1) * color.reshape(1, 1, -1)
    ax.imshow(mask_image)

def show_box(box, ax):
    x0, y0 = box[0], box[1]
    w, h = box[2] - box[0], box[3] - box[1]
    ax.add_patch(
        plt.Rectangle((x0, y0), w, h, edgecolor="blue", facecolor=(0, 0, 0, 0), lw=2)
    )

for step, (image, mask_binary, bboxes, img_name) in enumerate(tr_dataloader):
    print(image.shape, mask_binary.shape, bboxes.shape)
    # show the example
    _, axs = plt.subplots(1, 2, figsize=(25, 25))
    idx = random.randint(0, image.size(0) - 1)  # Update this line to get a valid index
    axs[0].imshow(image[idx].cpu().permute(1, 2, 0).numpy())
    show_mask(mask_binary[idx].cpu().numpy()[0], axs[0])  # Passing the 2D mask to show_mask
    show_box(bboxes[idx].numpy(), axs[0])
    axs[0].axis("off")
    # set title
    axs[0].set_title(img_name[idx])
    idx = random.randint(0, image.size(0) - 1)  # Update this line to get a valid index
    axs[1].imshow(image[idx].cpu().permute(1, 2, 0).numpy())
    show_mask(mask_binary[idx].cpu().numpy()[0], axs[1])  # Passing the 2D mask to show_mask
    show_box(bboxes[idx].numpy(), axs[1])
    axs[1].axis("off")
    # set title
    axs[1].set_title(img_name[idx])
    # plt.show()
    plt.subplots_adjust(wspace=0.01, hspace=0)
    plt.savefig("./data_sanitycheck.png", bbox_inches="tight", dpi=300)
    plt.close()
    break



number of images: 242
torch.Size([1, 3, 1024, 1024]) torch.Size([1, 1, 1024, 1024]) torch.Size([1, 4])


In [69]:
# !wget https://dl.fbaipublicfiles.com/segment_anything/sam_vit_b_01ec64.pth

In [70]:
class neuroSAM(nn.Module):
    def __init__(
        self,
        image_encoder,
        mask_decoder,
        prompt_encoder,
    ):
        super().__init__()
        self.image_encoder = image_encoder
        self.mask_decoder = mask_decoder
        self.prompt_encoder = prompt_encoder
        # freeze prompt encoder

        for param in self.prompt_encoder.parameters():
            param.requires_grad = False

        for param in self.image_encoder.parameters():
            param.requires_grad = False

    def forward(self, image, box):
        image_embedding = self.image_encoder(image)  # (B, 256, 64, 64)
        # do not compute gradients for prompt encoder
        with torch.no_grad():
            box_torch = torch.as_tensor(box, dtype=torch.float32, device=image.device)
            if len(box_torch.shape) == 2:
                box_torch = box_torch[:, None, :]  # (B, 1, 4)

            sparse_embeddings, dense_embeddings = self.prompt_encoder(
                points=None,
                boxes=box_torch,
                masks=None,
            )

        # print("Shape of image_embedding:", image_embedding.shape)
        # print("Shape of dense_prompt_embeddings:", dense_embeddings.shape)
    

        low_res_masks, iou_predictions = self.mask_decoder(
            image_embeddings=image_embedding,  # (B, 256, 64, 64)
            image_pe=self.prompt_encoder.get_dense_pe(),  # (1, 256, 64, 64)
            sparse_prompt_embeddings=sparse_embeddings,  # (B, 2, 256)
            dense_prompt_embeddings=dense_embeddings,  # (B, 256, 64, 64)
            multimask_output=False,
        )

        ori_res_masks = F.interpolate(
            low_res_masks,
            size=(image.shape[2], image.shape[3]),
            mode="bilinear",
            align_corners=False,
        )
        return ori_res_masks

In [71]:
# ls -a


In [72]:
model = sam_model_registry[model_type](checkpoint=checkpoint)

In [73]:
neurosam_model = neuroSAM(
        image_encoder=model.image_encoder,
        mask_decoder=model.mask_decoder,
        prompt_encoder=model.prompt_encoder,
    ).to(device)

neurosam_model.train()

neuroSAM(
  (image_encoder): ImageEncoderViT(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
    )
    (blocks): ModuleList(
      (0-11): 12 x Block(
        (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=768, out_features=2304, bias=True)
          (proj): Linear(in_features=768, out_features=768, bias=True)
        )
        (norm2): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
        (mlp): MLPBlock(
          (lin1): Linear(in_features=768, out_features=3072, bias=True)
          (lin2): Linear(in_features=3072, out_features=768, bias=True)
          (act): GELU(approximate='none')
        )
      )
    )
    (neck): Sequential(
      (0): Conv2d(768, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (1): LayerNorm2d()
      (2): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (3): LayerNorm2d()
 

In [74]:
print(
        "Number of total parameters: ",
        sum(p.numel() for p in neurosam_model.parameters()),
    )

Number of total parameters:  93735472


In [75]:
print(
        "Number of trainable parameters: ",
        sum(p.numel() for p in neurosam_model.parameters() if p.requires_grad),

    )

Number of trainable parameters:  4058340


In [76]:
img_mask_encdec_params = neurosam_model.mask_decoder.parameters()


In [77]:
optimizer = torch.optim.AdamW(
        img_mask_encdec_params, lr=0.0001, weight_decay=0.01
    )


In [78]:
seg_loss = monai.losses.DiceLoss(sigmoid=True, squared_pred=True, reduction="mean")


In [79]:
ce_loss = nn.BCEWithLogitsLoss(reduction="mean")


In [80]:
print("Number of training samples: ", len(tr_dataset))


Number of training samples:  242


In [81]:
!pip install -q wandb

In [82]:
import wandb

In [83]:
if use_wandb:
    import wandb

    wandb.login()
    wandb.init(
        project= task_name,
        config={
            "lr": lr,
            "batch_size": batch_size,
            "data_path": data_path,
            "model_type": model_type,
        },
    )

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


In [84]:
run_id = datetime.now().strftime("%Y%m%d-%H%M")
model_save_path = join(work_dir, task_name + "-" + run_id)
device = torch.device(device)

In [85]:
os.makedirs(model_save_path, exist_ok=True)

In [86]:
if resume is not None:
    if os.path.isfile(resume):
        checkpoint = torch.load(resume, map_location=device)
        start_epoch = checkpoint["epoch"] + 1
        optimizer.load_state_dict(checkpoint["optimizer"])

if use_amp:
    scaler = torch.cuda.amp.GradScaler()

In [87]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print("GO Ahead")

GO Ahead


In [88]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime! e/acc')

Your runtime has 403.7 gigabytes of available RAM

You are using a high-RAM runtime! e/acc


In [ ]:
for epoch in range(start_epoch, num_epochs):
    epoch_loss = 0
    for step, (image, gt2D, boxes, _) in enumerate(tqdm(tr_dataloader)):
        optimizer.zero_grad()
        boxes_np = boxes.detach().cpu().numpy()
        image, gt2D = image.to(device), gt2D.to(device)
        if use_amp:
            ## AMP
            with torch.autocast(device_type="cuda", dtype=torch.float16):
                neurosam_pred = neurosam_model(image, boxes_np)
                loss = seg_loss(neurosam_pred, gt2D) + ce_loss(
                    neurosam_pred, gt2D.float()
                )
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()
        else:
            neurosam_pred = neurosam_model(image,boxes_np)
            loss = seg_loss(neurosam_pred, gt2D) + ce_loss(neurosam_pred, gt2D.float())
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()

        epoch_loss += loss.item()
        iter_num += 1

    epoch_loss /= step
    losses.append(epoch_loss)
    if use_wandb:
        wandb.log({"epoch_loss": epoch_loss})
    print(
        f'Time: {datetime.now().strftime("%Y%m%d-%H%M")}, Epoch: {epoch}, Loss: {epoch_loss}'
    )
    ## save the latest model
    checkpoint = {
        "model": neurosam_model.state_dict(),
        "optimizer": optimizer.state_dict(),
        "epoch": epoch,
    }
    torch.save(checkpoint, join(model_save_path, "neurosam_model_latest.pth"))
    ## save the best model
    if epoch_loss < best_loss:
        best_loss = epoch_loss
        checkpoint = {
            "model": neurosam_model.state_dict(),
            "optimizer": optimizer.state_dict(),
            "epoch": epoch,
        }
        torch.save(checkpoint, join(model_save_path, "neurosam_model_best.pth"))

    # %% plot loss
    plt.plot(losses)
    plt.title("Dice + Cross Entropy Loss")
    plt.xlabel("Epoch")
    plt.ylabel("Loss")
    plt.savefig(join(model_save_path, task_name + "train_loss.png"))
    plt.close()

100%|██████████| 242/242 [01:39<00:00,  2.43it/s]


Time: 20240513-0551, Epoch: 0, Loss: 0.6663818996229607


100%|██████████| 242/242 [01:34<00:00,  2.57it/s]


Time: 20240513-0553, Epoch: 1, Loss: 0.5133766145812525


100%|██████████| 242/242 [01:34<00:00,  2.55it/s]


...

100%|██████████| 242/242 [01:38<00:00,  2.46it/s]


Time: 20240513-0820, Epoch: 93, Loss: 0.15404502790869518


100%|██████████| 242/242 [01:39<00:00,  2.42it/s]


Time: 20240513-0822, Epoch: 94, Loss: 0.1517198226217536


 98%|█████████▊| 236/242 [01:36<00:02,  2.46it/s]